# Unsuppervised Learning

In [20]:
#importar dados já processados na meta 1
import pandas as pd
import numpy as np
wines = pd.read_csv('merged.csv')

#binaryze values
wines['Quality'] = np.where(wines.Quality >= 7, 1, 0)

X = wines.drop(columns='Quality')
y = wines['Quality']


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# X_scaled_nparray = StandardScaler().fit_transform(X)
# X_scaled = pd.DataFrame(X_scaled_nparray, index=X.index, columns=X.columns)

#split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [21]:
# without clustering
log_reg = RandomForestClassifier(n_estimators=30, max_depth=9)
log_reg.fit(X_train, y_train)

print("train score:" ,log_reg.score(X_train, y_train))
print("test score:" ,log_reg.score(X_test, y_test))

train score: 0.9130325814536341
test score: 0.8368421052631579


In [22]:
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
pipeline = Pipeline([
    # como existem 2 possibilidades usamos 2 clusters
    ('kmeans', KMeans(n_clusters=2)),
    ('log_reg', RandomForestClassifier(n_estimators=30, max_depth=9))
])

pipeline.fit(X_train, y_train)

print("train score:" ,pipeline.score(X_train, y_train))
print("test score:" ,pipeline.score(X_test, y_test))

train score: 0.8145363408521303
test score: 0.8037593984962406


In [23]:
# agora considesando o possivel melhor numero de clusters
from sklearn.model_selection import GridSearchCV
param_grid = dict(kmeans__n_clusters=range(2, 16))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train, y_train)

Fitting 3 folds for each of 14 candidates, totalling 42 fits
[CV] END ...............................kmeans__n_clusters=2; total time=   0.4s
[CV] END ...............................kmeans__n_clusters=2; total time=   0.2s
[CV] END ...............................kmeans__n_clusters=2; total time=   0.2s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.1s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.1s
[CV] END ...............................kmeans__n_clusters=3; total time=   0.3s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.2s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.3s
[CV] END ...............................kmeans__n_clusters=4; total time=   0.2s
[CV] END ...............................kmeans__n_clusters=5; total time=   0.2s
[CV] END ...............................kmeans__n_clusters=5; total time=   0.2s
[CV] END ...............................kmeans__

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('kmeans', KMeans(n_clusters=2)),
                                       ('log_reg',
                                        RandomForestClassifier(max_depth=9,
                                                               n_estimators=30))]),
             param_grid={'kmeans__n_clusters': range(2, 16)}, verbose=2)

In [24]:
grid_clf.best_params_

{'kmeans__n_clusters': 7}

In [25]:
grid_clf.best_score_

0.8075187969924812